In [37]:
import os
import sys
sys.path.append(os.path.join("..", "..", "..", "build", "lib"))
from TbGAl.metric import EuclideanMetric
from TbGAl.multivector import *
from math import sin, cos, atan2, pi
from TbGAl.sparse_tensor import *
import numpy as np
import itertools

In [2]:
set_device(1)

In [3]:
DIMS = 10

In [4]:
Multivector.set_N(DIMS)

In [5]:
generate_T(EuclideanMetric())

In [6]:
e1 = e(1)
e2 = e(2)
e3 = e(3)
e4 = e(4)
e5 = e(5)
e6 = e(6)
e7 = e(7)
e8 = e(8)

# Objetivo: Calcular AB diretamente sobre os fatores

#### Sendo A = e1^e3^e4 - e1^e3^e5 + e2^e3^e4 - e2^e3^e5

In [7]:
A = [0.70710678 * e1 + 0.70710678 * e2, e3, 0.70710678 * e4 - 0.70710678 * e5]
A

[0.707107e(1)+0.707107e(2), e(3), 0.707107e(4)-0.707107e(5)]

In [81]:
def process_factors(factors_A, factor_B):
    new_factors_A = []
    inv_factor_B = INVERSE(factor_B)
    new_factor = GP(factors_A[0], inv_factor_B)
#     if new_factor.getGrade() == 2:
#         new_factors_A.append(process_factors(fact_versor(new_factor), factor_B))
#     else:
    new_factors_A.append(new_factor)        
    for i in range(1, len(A)):
#         print("fai: ", factors_A[i])
        new_factor = GP(GP(factor_B, factors_A[i]), inv_factor_B)
#         if new_factor.getGrade() == 2:
#             new_factors_A.append(process_factors(fact_versor(new_factor), factor_B))
#         else:
        new_factors_A.append(new_factor)
    
    return new_factors_A
#     return list(itertools.chain.from_iterable(new_factors_A))

In [107]:
def geometric_product_fact(factors_A, factors_B):
    new_factors = factors_A
#     print("NF: ", new_factors)
    count = 0
    for f in factors_B:
#         print(new_factors)
        newest_factors = []
        print("count: ", count)
        print("NF: ", new_factors)
        for k in process_factors(new_factors, f):
            print("K: ", k)
            if k.getGrade() == 2:
                newest_factors = newest_factors + fact_versor(k)[:-1]
            else:
                newest_factors.append(k)
            print("\t", newest_factors)
        new_factors = newest_factors
        count = count + 1
    return new_factors

In [108]:
geometric_product_fact(A, B)

count:  0
NF:  [0.707107e(1)+0.707107e(2), e(3), 0.707107e(4)-0.707107e(5)]
K:  0.707107e(1)^e(6)+0.707107e(2)^e(6)
	 [e(6), -0.707107e(1)-0.707107e(2)]
K:  -1.000000e(3)
	 [e(6), -0.707107e(1)-0.707107e(2), -1.000000e(3)]
K:  -0.707107e(4)+0.707107e(5)
	 [e(6), -0.707107e(1)-0.707107e(2), -1.000000e(3), -0.707107e(4)+0.707107e(5)]
count:  1
NF:  [e(6), -0.707107e(1)-0.707107e(2), -1.000000e(3), -0.707107e(4)+0.707107e(5)]
K:  e(6)^e(7)
	 [e(7), -1.000000e(6)]
K:  0.707107e(1)+0.707107e(2)
	 [e(7), -1.000000e(6), 0.707107e(1)+0.707107e(2)]
K:  e(3)
	 [e(7), -1.000000e(6), 0.707107e(1)+0.707107e(2), e(3)]
count:  2
NF:  [e(7), -1.000000e(6), 0.707107e(1)+0.707107e(2), e(3)]
K:  e(7)^e(8)
	 [e(8), -1.000000e(7)]
K:  e(6)
	 [e(8), -1.000000e(7), e(6)]
K:  -0.707107e(1)-0.707107e(2)
	 [e(8), -1.000000e(7), e(6), -0.707107e(1)-0.707107e(2)]


[e(8), -1.000000e(7), e(6), -0.707107e(1)-0.707107e(2)]

In [69]:
A

[0.707107e(1)+0.707107e(2), e(3), 0.707107e(4)-0.707107e(5)]

In [70]:
process_factors(A, e6)

fai:  -0.707107e(1)-0.707107e(2)
fai:  1.000000
fai:  e(3)
fai:  0.707107e(4)-0.707107e(5)


[1.000000,
 0.707107e(1)+0.707107e(2),
 1.000000,
 -1.000000e(3),
 -0.707107e(4)+0.707107e(5)]

In [71]:
process_factors(process_factors(A, e6), e7)

fai:  -0.707107e(1)-0.707107e(2)
fai:  1.000000
fai:  e(3)
fai:  0.707107e(4)-0.707107e(5)
fai:  0.707107e(1)+0.707107e(2)
fai:  1.000000


[e(7), -0.707107e(1)-0.707107e(2), 1.000000]

### Situação 1: B possui dependência linear com alguns fatores de A

#### B = e1+e2

In [15]:
B = [0.707107 * e1 + 0.707107 * e2, e3]
B

[0.707107e(1)+0.707107e(2), e(3)]

### Situação 2: B não possui dependência linear com alguns fatores de A

#### B = 1 + e6

In [72]:
B = [e6, e7, e8]